<a href="https://colab.research.google.com/github/risha-gandhi/bootstrapping_housing_data/blob/main/Boostrapping_on_housing_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this exercise, we will get more comfortable working with Pandas and start working with SKLearn - one of the main python machine learning libraries. We will be working with a dataset on homes in King County from 2015. The data is available here: https://www.kaggle.com/harlfoxem/housesalesprediction  

We are interested in predicting the price of a house based on various features about the home.

Please skim/read the introductory tutorial on SKLearn [here](https://jakevdp.github.io/PythonDataScienceHandbook/05.02-introducing-scikit-learn.html) (up to section 5) and answer the following questions.

In [ ]:
import pandas as pd
import numpy as np
from sklearn import linear_model, preprocessing, impute
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, mean_squared_error
import seaborn as sns


In [ ]:
data = pd.read_csv('kc_house_data.csv')
print(data.head()) 
data.describe()

           id             date     price  ...     long  sqft_living15  sqft_lot15
0  7129300520  20141013T000000  221900.0  ... -122.257           1340        5650
1  6414100192  20141209T000000  538000.0  ... -122.319           1690        7639
2  5631500400  20150225T000000  180000.0  ... -122.233           2720        8062
3  2487200875  20141209T000000  604000.0  ... -122.393           1360        5000
4  1954400510  20150218T000000  510000.0  ... -122.045           1800        7503

[5 rows x 21 columns]


,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,2.161300e+04,2.161300e+04,21613.000000,21613.000000,21613.000000,2.161300e+04,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000
mean,4.580302e+09,5.400881e+05,3.370842,2.114757,2079.899736,1.510697e+04,1.494309,0.007542,0.234303,3.409430,7.656873,1788.390691,291.509045,1971.005136,84.402258,98077.939805,47.560053,-122.213896,1986.552492,12768.455652
std,2.876566e+09,3.671272e+05,0.930062,0.770163,918.440897,4.142051e+04,0.539989,0.086517,0.766318,0.650743,1.175459,828.090978,442.575043,29.373411,401.679240,53.505026,0.138564,0.140828,685.391304,27304.179631
min,1.000102e+06,7.500000e+04,0.000000,0.000000,290.000000,5.200000e+02,1.000000,0.000000,0.000000,1.000000,1.000000,290.000000,0.000000,1900.000000,0.000000,98001.000000,47.155900,-122.519000,399.000000,651.000000
25%,2.123049e+09,3.219500e+05,3.000000,1.750000,1427.000000,5.040000e+03,1.000000,0.000000,0.000000,3.000000,7.000000,1190.000000,0.000000,1951.000000,0.000000,98033.000000,47.471000,-122.328000,1490.000000,5100.000000
50%,3.904930e+09,4.500000e+05,3.000000,2.250000,1910.000000,7.618000e+03,1.500000,0.000000,0.000000,3.000000,7.000000,1560.000000,0.000000,1975.000000,0.000000,98065.000000,47.571800,-122.230000,1840.000000,7620.000000
75%,7.308900e+09,6.450000e+05,4.000000,2.500000,2550.000000,1.068800e+04,2.000000,0.000000,0.000000,4.000000,8.000000,2210.000000,560.000000,1997.000000,0.000000,98118.000000,47.678000,-122.125000,2360.000000,10083.000000
max,9.900000e+09,7.700000e+06,33.000000,8.000000,13540.000000,1.651359e+06,3.500000,1.000000,4.000000,5.000000,13.000000,9410.000000,4820.000000,2015.000000,2015.000000,98199.000000,47.777600,-121.315000,6210.000000,871200.000000


Question 1. 

a) How many houses are in the dataset? 

b) How many columns are there? How many should be included as part of our training model?

Answer: There are 21 columns in the dataset and we would include 20 columns (excluding 'id') in the training model. I also checked for correlation between the variables as we would exclude columns of the variables that are highly correlated to avoid redundant variables that are providing the same information, but didn't find any high correlations.

In [ ]:
# Part a):

print(len(data.id.unique()))

# Part b):
print(len(data.columns))

data.corr()['price'].sort_values(ascending=False)


21436
21


price            1.000000
sqft_living      0.702035
grade            0.667434
sqft_above       0.605567
sqft_living15    0.585379
bathrooms        0.525138
view             0.397293
sqft_basement    0.323816
bedrooms         0.308350
lat              0.307003
waterfront       0.266369
floors           0.256794
yr_renovated     0.126434
sqft_lot         0.089661
sqft_lot15       0.082447
yr_built         0.054012
condition        0.036362
long             0.021626
id              -0.016762
zipcode         -0.053203
Name: price, dtype: float64

a) Make a histogram of the house prices.

b) Using plt.scatter, plot the house prices against sq_ft living. What do you notice?
 

In [ ]:
# Part a)
plt.hist(data.price)
plt.show()

# Part b)
plt.scatter(data.price, data.sqft_living)
plt.xlabel("Price")
plt.ylabel("Sqft_living")
plt.show()
# The price of a house keeps increasing as it's square feet increases (positive correlation)


Now we are going to make a predictive linear model. First we will pull out the dependent variable column, ``price``, and we will also create a list of features we will use in our model. We will also pull out the predictive variables ``x``.

In [ ]:
y = data['price']
features = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 
            'floors', 'zipcode',  'condition', 'grade', 'waterfront', 
            'view', 'sqft_above', 'sqft_basement',  'yr_built', 'yr_renovated', 
            'lat', 'long', 'sqft_living15']
x = data[features]

a) Training a linear regression model that uses ``x`` as the features, and ``y`` as the dependent variable. 

b) Computing R2 of the model and intercept

In [ ]:
model = linear_model.LinearRegression()

model.fit(x,y)

r2 = model.score(x,y)
print("R-Squared Value: ",r2)

print("Model intercept: ", model.intercept_)

Question 4. ``model.coef_`` is the array of model coefficients. We are interested in building a confidence interval on ``model.coef_[2]`` which corresponds to the ``sqft_living`` feature. 

To do this I am using the bootstrap. 

a) The code randomly resamples the training data 50 times. Each time I resample, I retrained my model and saved the coefficient of ``sqft_living``. Then plotted the sampling distribution and returned a 95% bootstrap confidence interval. 

b) What would have happened if I had assumed normality and made a confidence interval based on the standard error? Which confidence interval do I trust better?

In [ ]:
# This function calculates the confidence interval based on some coefficient array and the alpha value
def CI_formula(coeffs, alpha):
  p_lower = ((1.0-alpha)/2.0) * 100
  lower = np.percentile(coeffs, p_lower)
  p_upper = (alpha+((1.0-alpha)/2.0)) * 100
  upper = np.percentile(coeffs, p_upper)
  return lower, upper

In [ ]:
coeffs = []
for i in range(50):
    resample_idxs = np.random.choice(len(data), len(data))
    y_resample = y[resample_idxs]
    x_resample = x.iloc[resample_idxs]

    #retrain/fit the model
    model = linear_model.LinearRegression().fit(x_resample,y_resample)
    # pick the sqft_living or model.coef_[2]
    sqft_living = model.coef_[2]
    # append that sqft_living in coeffs
    coeffs.append(sqft_living)

sns.distplot(coeffs)


#Calculate upper and lower bounds of Confidence Interval(95%)
alpha = 0.95
lower, upper = CI_formula(coeffs, alpha)

print("95% Confidence Interval is: ", round(lower,3), ",", round(upper,3)) 

plt.axvline(lower, color='k', linestyle='dashed', linewidth=1)
plt.axvline(upper, color='k', linestyle='dashed', linewidth=1)
plt.show()

#Confidence interval- Assuming normality
import scipy.stats as sp
CI = sp.norm.interval(alpha = 0.95, loc = np.mean(coeffs), scale = sp.sem(coeffs))
lower_new, upper_new = sp.norm.interval(alpha = 0.95, loc = np.mean(coeffs), scale = sp.sem(coeffs))
print("\n\n95% Confidence Interval assuming normality: ", CI,"\n\n") 

print("95% Confidence Interval assuming normality: ", round(lower_new,3), ",", round(upper_new,3)) 

"""
On computing the confidence interval assuming normality, it appears to be quite different and restrictive from the interval computed using the bootstrapped 
distribution. We also experimented by re-sampling 100 times instead of 50 and the distribution looks more bimodal and less like a normal distribution.

Hence, we would trust the interval from the bootstrapped distribution more. Since its confidence interval is wider than the one assuming normality and 
calculated using the formula, it is more accurate and likely to contain the true population mean when resampled a number of times. We cannot 
assume normality here, because if we did, we would have gotten the same confident interval using the formula and from the one assuming normality, 
which was proven to be false here.
"""
